In [28]:
import os.path
import torch

PATH_TO_PRETRAINED = '/home/vika/comparative-dialogue/comparative-dialogue/external_pretrained_models/'
MODEL_NAMES = ['model5_tagger.hdf5']

def load(checkpoint_fn, gpu=-1):
    if not os.path.isfile(PATH_TO_PRETRAINED + checkpoint_fn):
        raise ValueError('Can''t find tagger in file "%s". Please, run the main script with non-empty \
                         "--save-best-path" param to create it.' % checkpoint_fn)
    tagger = torch.load(PATH_TO_PRETRAINED + checkpoint_fn)
    tagger.gpu = gpu

    tagger.word_seq_indexer.gpu = gpu # hotfix
    tagger.tag_seq_indexer.gpu = gpu # hotfix
    if hasattr(tagger, 'char_embeddings_layer'):# very hot hotfix
        tagger.char_embeddings_layer.char_seq_indexer.gpu = gpu # hotfix
    tagger.self_ensure_gpu()
    return tagger

In [29]:
!nvidia-smi

Sun Nov  3 23:12:58 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.87                 Driver Version: 390.87                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 55%   77C    P2   135W / 250W |   5926MiB / 11170MiB |     27%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 51%   72C    P2    63W / 250W |  10812MiB / 11178MiB |      7%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [30]:
!pwd

/home/vika/comparative-dialogue/comparative-dialogue


In [31]:
def create_sequence_from_sentence(str_sentences):
    return [str_sentence.lower().split() for str_sentence in str_sentences]

In [32]:
words = create_sequence_from_sentence(['what is better amazon or itunes', 'who are better mice or rats', 'what is easier to make bread o pizza', 'what is better for startap python or mathlab', 'what is better perl or python', 'what is better cuda or opencl', 'what is better gamecube or ps2', 'what is preferable beer or milk', 'beer is more preferable than milk becouse beer is better absorbed', 'mices are more cute than rats', 'python is better than matlab for startup becouse it is free'])

In [33]:
import sys
sys.path.append('/home/vika/NER_RNN/targer')
from src.factories.factory_tagger import TaggerFactory

for MODEL_NAME in MODEL_NAMES:
    print (MODEL_NAME)
    model = TaggerFactory.load(PATH_TO_PRETRAINED + MODEL_NAME)
    model.cuda(device=1)
    model.gpu = 1
    tags = model.predict_tags_from_words(words)
    for ind, elem in enumerate(words):
        print (elem)
        print (tags[ind])
        print ('\n')

model5_tagger.hdf5


['what', 'is', 'better', 'amazon', 'or', 'itunes']
['O', 'O', 'O', 'B-OTHOBJ', 'O', 'B-OTHOBJ']


['who', 'are', 'better', 'mice', 'or', 'rats']
['O', 'O', 'O', 'O', 'O', 'O']


['what', 'is', 'easier', 'to', 'make', 'bread', 'o', 'pizza']
['O', 'O', 'O', 'O', 'O', 'B-ASPOBJ', 'O', 'B-OTHOBJ']


['what', 'is', 'better', 'for', 'startap', 'python', 'or', 'mathlab']
['O', 'O', 'O', 'O', 'O', 'B-OTHOBJ', 'O', 'O']


['what', 'is', 'better', 'perl', 'or', 'python']
['O', 'O', 'O', 'O', 'O', 'B-OTHOBJ']


['what', 'is', 'better', 'cuda', 'or', 'opencl']
['O', 'O', 'O', 'O', 'O', 'O']


['what', 'is', 'better', 'gamecube', 'or', 'ps2']
['O', 'O', 'O', 'B-OTHOBJ', 'O', 'O']


['what', 'is', 'preferable', 'beer', 'or', 'milk']
['O', 'O', 'O', 'B-ASPOBJ', 'O', 'B-OTHOBJ']


['beer', 'is', 'more', 'preferable', 'than', 'milk', 'becouse', 'beer', 'is', 'better', 'absorbed']
['B-ASPOBJ', 'O', 'O', 'O', 'O', 'B-ASPOBJ', 'O', 'O', 'O', 'O', 'O']


['mices', 'are', 'more', 'cute'

/home/vika/.local/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'src.models.tagger_birnn_crf.TaggerBiRNNCRF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [25]:
tags

[['O', 'O', 'O', 'B-ASPOBJ', 'O', 'B-OTHOBJ'],
 ['O', 'O', 'O', 'B-ASPOBJ', 'O', 'B-OTHOBJ'],
 ['O', 'O', 'O', 'O', 'O', 'B-OTHOBJ', 'O', 'B-OTHOBJ'],
 ['O', 'O', 'O', 'O', 'O', 'B-ASPOBJ', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'B-OTHOBJ'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'B-ASPOBJ', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'B-OTHOBJ']]

In [8]:
def get_objects(list_words, list_tags):
    obj_list = []
    for ind, elem in enumerate(list_tags):
        if elem == 'B-OTHOBJ' or elem == 'B-ASPOBJ':
            obj_list.append(list_words[ind])
    return obj_list

In [9]:
objects_list = []
for elem in list(zip(words, tags)):
    objects = get_objects(elem[0], elem[1])
    if len(objects) >= 2:
        objects_list.append((objects[0], objects[1]))
    else:
        print("We have %d objects to compare %s" %(len(objects), objects))
    

In [10]:
from util import get_response
from generation.generation import diviner

for obj0, obj1 in objects_list:
    response = get_response(obj0, obj1, False)
    response_json = response.json()
    Merlin = diviner()
    Merlin.create_from_json(response_json)
    Merlin.generate_advice()

winnder: amazon  other: itunes
acpect winner  faster, greater, deeper, faster for me, easier to interface here
acpect other  worse for consumers, easier, easier to work with, easier to navigate, easier to use store, better for random browsing and searching, cleaner


In this context, amazon are preferable to itunes, as it is faster, greater, deeper, faster for me, easier to interface here.
 Itunes are worse for consumers, easier, easier to work with, easier to navigate, easier to use store, better for random browsing and searching, cleaner
winnder: rat  other: mouse
acpect winner  greater, easier to work with, bigger, easier, differentiation, easier to study diseases in rats, larger, easier to isolate and culture
acpect other  quicker, faster, shower, easier to pick up, easier to trap, richer


rat mouse
acpect winner  greater, easier to work with, bigger, easier, differentiation, easier to study diseases in rats, larger, easier to isolate and culture
acpect other  quicker, faster, sho

In [13]:
!pwd

/home/vika/comparative-dialogue/comparative-dialogue


In [104]:
import pandas as pd
fixed_df = pd.read_csv('/home/vika/comparative-dialogue/comparative-dialogue/datasets/marco_data/marco_CQA.csv')

In [105]:
fixed_df.head()

,Unnamed: 0,question,short_answ,long_answ,explanation
0,0,can a persons credit get better aftef a year?,['no'],"[""No, a person's credit can not get better aft...",You may want to apply for a secured credit car...
1,1,is project m better than melee,"['No', 'Yes']","['No, project M is not better than melee.']",The logo for Project M. Project M is a gamepla...
2,2,is it better to have default instance or named...,['Default'],['It is better to have a default instance.'],If a default instance is listening on a port o...
3,3,is it better to be the first or last to interv...,['Yes'],"['yes, it is better to be the first or last to...",Suppose you had the job to interview students ...
4,4,what is better memory foam or gel memory foam,['Gel memory foam'],['Gel memory foam is better than a memory foam.'],Whether you go with a traditional memory foam ...


In [106]:
zip_list = list(zip(fixed_df['question'].values, fixed_df['long_answ'].values, fixed_df['explanation'].values))

In [107]:
for elem in zip_list:
    print('\n')
    print (elem[0])
    print (elem[1])
    print (elem[2])



can a persons credit get better aftef a year?
["No, a person's credit can not get better after a year."]
You may want to apply for a secured credit card that can be used in moderation and paid off each month. Secured credit cards use your savings account as collateral for the credit limit and are easier to be approved for than a standard credit card.


is project m better than melee
['No, project M is not better than melee.']
The logo for Project M. Project M is a gameplay modification of Super Smash Bros. Brawl designed to make the gameplay more closely resemble that of Super Smash Bros. Melee mainly, and also Super Smash Bros. to a lesser extent.This means that the speed of gameplay has been generally increased, with many Melee mechanics being restored.inal Destination. Reverted to Melee 's version to fix issues with getting stuck underneath Brawl 's edges when returning (despite a Project M fix for Brawl 's Final Destination already being available). Also replaces Opening (Super S